In [ ]:
from airfoil import Airfoil, Decomposer
from airfoil.cnc import GCodeBuilder as gcb, CNC
from airfoil.util.shapely_helpers import plot_shapely_directional
from airfoil.util.linestring_helpers import (
    split_and_roll_at_top_right,
    ensure_closed,
    deflection_angle_padded,
    deflection_angle,
    split_and_roll,
    resample_long_segments
)
from airfoil.wing import (
    auto_interpolate
)
from airfoil.util.array_helpers import map_to_range

import shapely as sh
import numpy as np

In [ ]:
af = Airfoil.from_naca_designation("0012", chord_length=100)
af.plot_raw()

In [ ]:
thickness = 10
taper     = 100
sh_top = sh.LineString(resample_long_segments(np.array([
    ( -5,  thickness),
    (  0,  thickness),
    ( taper, 25),
    ( taper, 30),
]),1))
sh_bottom = sh.LineString(resample_long_segments(np.array([
    (-5,-thickness),
    ( 0,-thickness),
    (taper,-25),
    (taper,-30),
]),1))

In [ ]:
thickness = 10
taper     = 100
sh_top = sh.LineString(resample_long_segments(np.array([
    ( -5,  thickness),
    (  0,  thickness),
    ( taper, 35),
    ( taper, 40),
]),1))
sh_bottom = sh.LineString(resample_long_segments(np.array([
    (-5,-thickness),
    ( 0,-thickness),
    (taper,-15),
    (taper,-30),
]),1))

In [ ]:
afp = sh.Polygon(af.with_rotation(180).with_translation((60,0)).points)
afp = sh.difference(
    afp,
    sh.box(
        -500,-500,
        -5  , 500
    )
).buffer(-0.1)
afc:np.ndarray = np.array(afp.exterior.coords)
afc = split_and_roll(afc, np.argmax(afc[:,1]-afc[:,0]))
afc=afc[1:]
afc=afc[::-1]
plot_shapely_directional([sh.LineString(afc),sh_top,sh_bottom],legend=["afc","top","bottom"])

In [ ]:
cnc = CNC()

In [ ]:
def compute_speed(curve:np.ndarray, min_mm_per_sec:float=110, max_mm_per_sec:float=180):
    return map_to_range(
        np.pad(deflection_angle(curve),(0,1),mode="edge"),
        max_mm_per_sec,
        min_mm_per_sec,
    )

In [ ]:
cnc.travel(0,0,0,0)

In [ ]:
cnc.relative()
cnc.travel(
    0,-2,
    0,-2,
)

In [ ]:
cnc.home()

In [ ]:
cnc.alarm_soft_reset()
cnc.alarm_clear()

In [ ]:
cnc.home()

In [ ]:

gc = (
    gcb()
    .set_position(0,0,0,0)
    .absolute()
    .set_current(1.85)
    .path_absolute(
        xyza     = np.tile(sh_top.coords,(1,2)),
        feedrate = compute_speed(sh_top.coords)
    )
    .travel(-5,30,-5,30)
    .path_absolute(
        xyza     = np.tile(sh_bottom.coords,(1,2)),
        feedrate = compute_speed(sh_bottom.coords)
    )
    .travel(-5,-30,-5,-30)
    .path_absolute(
        xyza     = np.tile(afc,(1,2)),
        feedrate = compute_speed(afc)
    )
    .set_current(0)
    .travel(-5,-25,-5,-25)
)
cnc.send_gcode_lines(gc.lines)
gc.lines

In [ ]:
af = Airfoil.from_naca_designation("23012", 200).with_rotation(4)
upper,lower = Decomposer().decompose(af)
sh_lead_in = sh.LineString(resample_long_segments([
    upper[0],
    (210,+37)
],5)).reverse()
sh_lower = sh.LineString(lower[::-1])
sh_upper = sh.LineString(upper[::-1])
sh_lead_out = sh.LineString(resample_long_segments([
    upper[0],
    (300,-37)
],5))
plot_shapely_directional([
    sh_lead_in,
    sh_lower,
    sh_upper,
    sh_lead_out,  
])
xyza_lead_in  = np.array(sh_lead_in.coords)
xyza_lower    = np.array(sh_lower.coords)
xyza_upper    = np.array(sh_upper.coords)
xyza_lead_out = np.array(sh_lead_out.coords)
plot_shapely_directional([sh.LineString(np.concat([xyza_lead_in,xyza_lower,xyza_upper,xyza_lead_out]))])

In [ ]:
cnc.alarm_soft_reset()
cnc.alarm_clear()

In [ ]:
cnc.relative()
cnc.travel(
    0,150,
    0,150,
)

In [ ]:
cnc.set_position(0,0,0,0)

In [ ]:
cnc.absolute()
cnc.travel(200,0,200,0)
cnc.travel(0,0,0,0)
cnc.travel(200,0,200,0)
cnc.travel(0,0,0,0)
cnc.travel(200,0,200,0)
cnc.travel(0,0,0,0)

In [ ]:
cnc.home()
cnc.set_position(0,0,0,0)

In [ ]:
cnc.relative()
cnc.travel(
    0,-5,
    0,-5,
)

In [ ]:
cnc.absolute()
cnc.travel(*np.tile(xyza_lead_in,(1,2))[0])

In [ ]:
cnc.serial.readall()

In [ ]:
cnc.serial.readall()
cnc.serial.readall()
cnc.serial.readall()
gc = (
    gcb()
    .set_position(0,0,0,0)
    .absolute()
    .set_current(1.85)
    .path_absolute(
        xyza=np.tile(xyza_lead_in,(1,2)),
        feedrate=compute_speed(xyza_lead_in),
    )
    .path_absolute(
        xyza=np.tile(xyza_lower,(1,2)),
        feedrate=compute_speed(xyza_lower),
    )
    .path_absolute(
        xyza=np.tile(xyza_upper,(1,2)),
        feedrate=compute_speed(xyza_upper),
    )
    .path_absolute(
        xyza=np.tile(xyza_lead_out,(1,2)),
        feedrate=compute_speed(xyza_lead_out),
    )
    .set_current(0)
)
cnc.send_gcode_lines(gc.lines,timeout_seconds=999)
gc.lines